In [1]:
import pennylane as qml
from pennylane import numpy as np

## G.1.1

Concept: in addition to the oracle $U_f$, we want a **diffusion operator** $D$. This $D$ should collect mass from positive amplitudes, invert mass in negative amplitudes, and redistribute the collected mass to those initially with negative amplitudes.

In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2] * len(combo))  # Index of solution
    my_array = np.identity(2 ** len(combo))  # Create the identity matrix
    my_array[index, index] = -1
    return my_array


@qml.qnode(dev)
def oracle_amp(combo):
    """Prepare the uniform superposition and apply the oracle.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The quantum state (amplitudes) after applying the oracle.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.broadcast(qml.Hadamard, wires=range(n_bits), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    return qml.state()


## G.1.2a

Concept: the diffusion operator: $D=2\ket{\psi}\bra{\psi}-I$, where $\ket{\psi}$ is the uniform superposition; this shifts a chunk of amplitude proportional to $1/\sqrt{N}$.

Concept: $G=DU_f$, the **Grover search**.

Geometrically, we may plot  the state vector on a 2-D plane, where, say, the vertical axis represents the amplitude of any solution component, and the horizontal axis represents the amplitude of any non-solution component. Then $U_f$ mirrors this state vector along the horizontal axis, and $D$ mirrors state vectors about the vector representing the uniform superposition $\ket{\psi}$.

Intuition: decompose the vector into two; one representing the uniform superposition $\ket{\psi}$, and another of all components orthogonal to it. One might think to decompose to $\ket{\psi}$ and its delta against the current state; in which case we expect that the delta contains

- negative amplitudes between $0$ and $-a$ inclusive in each non-solution component, where $a$ is the amplitude in that component in the uniform superposition, and
- in each solution component, the delta contains a large positive amplitude equal to the negation of the post-$U_f$ negated amplitude, plus the typical amplitude in the uniform superposition.

However, this delta vector is not orthogonal.

Instead, assuming that the $\ket{\psi}$ component has positive magnitude, then the orthogonal component should still have some positive mass in the non-solution components, and negative mass in the solution components equal to the mass in the non-solution components. This is because orthogonality is equivalent to the dot product against $\ket{\psi}$ being zero, which we can achieve with this. The diffusion component flips this orthogonal vector, so that

- A non-solution component not only loses mass from part of it going to the orthogonal vector, it then has exactly that amount further subtracted from it by adding the negated orthogonal vector again.
- A solution component gets its negative mass and more in the orthogonal component, which is flipped back to a positive value and further added to its amplitude in the $\ket{\psi}$ component.

But for sufficiently low amplitudes in the non-solution components, or negative amplitudes, decomposition requires a negative magnitude in the $\ket{\psi}$ component, so that the orthogonal component has sufficient positive amplitude in the non-solution components to cancel out the amplitude in the solution component. Once this happens the absolute amplitudes start to equalize again in magnitude under Grover transforms. In this regime $U_f$ negates the solution vectors, and splitting this result into $\ket{\psi}$ component and an orthogonal vector, the $\ket{\psi}$ component has negative amplitude, and the orthogonal component contains positive amplitudes in the non-solution components, and negative amplitudes in the solution component. This gets flipped, so summing back together, the non-solution components get more negative amplitude added to them. The solution components have their amplitude split among the $\ket{\psi}$ vector and the orthogonal vector; then the orthorganal vector part gets flipped, is greater in magnitude than the $\ket{\psi}$ vector part, and so ends up positive with a diminished amplitude in the end.

In [ ]:
n_bits = 4


def diffusion_matrix():
    """Return the diffusion matrix.

    Returns:
        array[float]: The matrix representation of the diffusion operator.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    l = 2 ** n_bits
    my_array = 2 ** (1-n_bits) * np.ones(l) - np.identity(l)
    return my_array


@qml.qnode(dev)
def difforacle_amp(combo):
    """Apply the oracle and diffusion matrix to the uniform superposition.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The quantum state (amplitudes) after applying the oracle
        and diffusion.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.broadcast(qml.Hadamard, wires=range(n_bits), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.QubitUnitary(diffusion_matrix(), wires=range(n_bits))
    return qml.state()


## G.1.2b

In [ ]:
@qml.qnode(dev)
def two_difforacle_amp(combo):
    """Apply the Grover operator twice to the uniform superposition.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The resulting quantum state.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.broadcast(qml.Hadamard, wires=range(n_bits), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.QubitUnitary(diffusion_matrix(), wires=range(n_bits))
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.QubitUnitary(diffusion_matrix(), wires=range(n_bits))
    return qml.state()
